<h2 style="color:dodgerblue"> 
ISEF 2024-25 Project
</h2>

<h4 style="color:white"> 
Quantum Error Correction Codes: Accuracy vs Time Complexity
</h4>

##### <span style="color: white;"> By: Sumer Chaudhary, BASIS Independent McLean
______________________
<h6 style="color:dodgerblue"> 
12/15/2024 - 
</h6>

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, generate_preset_pass_manager, transpile
from qiskit.visualization import plot_histogram, plot_circuit_layout
from qiskit.quantum_info import Statevector, Operator
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.circuit import IfElseOp
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService

service=QiskitRuntimeService(channel="ibm_quantum")

In [2]:
def cxx(qc, c, q1, q2):
    qc.cx(c, q1)
    qc.cx(c, q2)

def cxxx(qc, c, q1, q2, q3):
    qc.cx(c, q1)
    qc.cx(c, q2)
    qc.cx(c, q3)

def cxxxx(qc, c, q1, q2, q3, q4): 
    qc.cx(c,q1)
    qc.cx(c,q2)
    qc.cx(c,q3)
    qc.cx(c,q4)

def czz(qc, c, q1, q2):
    qc.cz(c, q1)
    qc.cz(c, q2)

def czzz(qc, c, q1, q2, q3):
    qc.cz(c, q1)
    qc.cz(c, q2)
    qc.cz(c, q3)

def czzzz(qc, c, q1, q2, q3, q4): 
    qc.cz(c,q1)
    qc.cz(c,q2)
    qc.cz(c,q3)
    qc.cz(c,q4)

def cxxzz(qc, c, q1, q2, q3, q4):
    qc.cx(c, q1)
    qc.cx(c, q2)
    qc.cz(c, q3)
    qc.cz(c, q4)

In [3]:
def encode_with_shors(qc, log_reg, stab_reg):
    cxx(qc, log_reg[0], stab_reg[2], stab_reg[5])

    qc.h(log_reg[0])
    qc.h(stab_reg[2])
    qc.h(stab_reg[5])

    cxx(qc, log_reg[0], stab_reg[0], stab_reg[1])

    cxx(qc, stab_reg[2], stab_reg[3], stab_reg[4])

    cxx(qc, stab_reg[5], stab_reg[6], stab_reg[7])

    qc.barrier()


def decode_with_shors(qc, log_reg, class_reg, stab_reg):
    cxx(qc, log_reg[0], stab_reg[0], stab_reg[1])

    cxx(qc, stab_reg[2], stab_reg[3], stab_reg[4])

    cxx(qc, stab_reg[5], stab_reg[6], stab_reg[7])

    qc.ccx(stab_reg[1], stab_reg[0], log_reg[0])
    qc.ccx(stab_reg[4], stab_reg[3], stab_reg[2])
    qc.ccx(stab_reg[7], stab_reg[6], stab_reg[5])

    qc.h(log_reg[0])
    qc.h(stab_reg[2])
    qc.h(stab_reg[5])

    cxx(qc, log_reg[0], stab_reg[2], stab_reg[5])

    qc.ccx(stab_reg[5], stab_reg[2], log_reg[0])

    qc.barrier()

In [4]:
def encode_with_steane(qc, log_q, stab_reg):
    cxx(qc, log_q, stab_reg[4], stab_reg[5])
    
    qc.h(stab_reg[:3])
        
    cxxx(qc, stab_reg[0], stab_reg[3], stab_reg[5], log_q)

    cxxx(qc, stab_reg[1], stab_reg[3], stab_reg[4], log_q)

    cxxx(qc, stab_reg[2], stab_reg[3], stab_reg[4], stab_reg[5])

    qc.barrier()

def steane_measure_syndrome(qc, log_q, stab_reg, anc_reg, class_reg):
    qc.h(anc_reg[:6])

    cxxxx(qc, anc_reg[0], stab_reg[0], stab_reg[4], stab_reg[5], log_q)
    cxxxx(qc, anc_reg[1], stab_reg[1], stab_reg[3], stab_reg[5], log_q)
    cxxxx(qc, anc_reg[2], stab_reg[2], stab_reg[3], stab_reg[4], stab_reg[5])
    
    czzzz(qc, anc_reg[3], stab_reg[0], stab_reg[4], stab_reg[5], log_q)
    czzzz(qc, anc_reg[4], stab_reg[1], stab_reg[3], stab_reg[5], log_q)
    czzzz(qc, anc_reg[5], stab_reg[2], stab_reg[3], stab_reg[4], stab_reg[5])

    qc.h(anc_reg)    
    qc.measure(anc_reg, class_reg)

    qc.barrier()


def steane_correct_errors(qc, log_q, stab_reg, class_reg):
    # Define correction gates using a mapping of the table values.
    def apply_correction(circuit, correction, qubit):
        if correction == "X":
            circuit.x(qubit)
        elif correction == "Z":
            circuit.z(qubit)
        elif correction == "Y":
            circuit.y(qubit)
        elif correction == "I":
            circuit.i(qubit) #does nothing, just to be there

    corrections = { # All of them are the Y errors, as the first half of the number would be the Z error, and the second, the X
    36: stab_reg[0],
    18: stab_reg[1],
    9:  stab_reg[2],
    54: stab_reg[3],
    45: stab_reg[4],
    63: stab_reg[5],
    27: log_q
    }

    # Iterate through all correction cases from the table.
    for decimal_value, qubit in corrections.items():
        # Prepare the classical condition for the current value.
        one_vals = []
        condition = [int(x) for x in format(decimal_value, '06b')]
        control_ops = []
        for i, value in enumerate(condition):
            if value==1:
                one_vals.append(i-len(one_vals))
            control_ops.append(qc.if_test((class_reg[i], value)))

        for i in range(len(one_vals)):
            if_test = control_ops.pop(one_vals[i])
            control_ops.append(if_test)

        
        one_vals = [] #clear the one_vals, as they are no longer correct
        
        for i in range(len(control_ops)): 
            if control_ops[i].condition[1]==1:
                one_vals.append(i)

        # Split the one_vals into Z and X checks for the Y condition.
        z_checks = one_vals[:len(one_vals) // 2]  # First half corresponds to Z checks.
        x_checks = one_vals[len(one_vals) // 2 : len(one_vals)]  # Second half corresponds to X checks.
        
        
        # Apply the correction for the matched condition.
        if z_checks[0] != 0:
            with control_ops[0]:
                with control_ops[1]:
                    if z_checks[0] != 2:
                        with control_ops[2]:
                            with control_ops[3]:
                                if z_checks[0] != 4: #if we reach this point, z_checks should always be 4, this is just for safety
                                    with control_ops[4]:
                                        with control_ops[5]:
                                            apply_correction(qc, "I", qubit)

                                else: #2 bits need to be one
                                    with control_ops[z_checks[0]] as else1:
                                        with control_ops[x_checks[0]] as else2:
                                            apply_correction(qc, "Y", qubit)
                                        with else2:
                                            apply_correction(qc, "Z", qubit)
                                    with else1:
                                        with control_ops[x_checks[0]]:
                                            apply_correction(qc, "X", qubit)
                                        
                    else: #4 bits need to be one
                        with control_ops[z_checks[0]] as else1:
                            with control_ops[z_checks[1]]:
                                with control_ops[x_checks[0]] as else2:
                                    with control_ops[x_checks[1]]:
                                        apply_correction(qc, "Y", qubit)
                                with else2:
                                    apply_correction(qc, "Z", qubit)    
                        with else1:
                            with control_ops[x_checks[0]]:
                                 with control_ops[x_checks[1]]:
                                    apply_correction(qc, "X", qubit)
                                     
        else: #All 6 bits need to be one
            with control_ops[z_checks[0]] as else1:
                with control_ops[z_checks[1]]:
                    with control_ops[z_checks[2]]:
                        with control_ops[x_checks[0]] as else2:
                            with control_ops[x_checks[1]]:
                                with control_ops[x_checks[2]]:
                                    apply_correction(qc, "Y", qubit)
                        with else2:
                            apply_correction(qc, "Z", qubit)
            with else1:
                with control_ops[x_checks[0]]:
                    with control_ops[x_checks[1]]:
                        with control_ops[x_checks[2]]:
                            apply_correction(qc, "X", qubit)


    qc.barrier()

    
def decode_with_steane(qc, log_q, stab_reg, out_q):
    qc.cx(stab_reg[0], out_q)
    qc.cx(stab_reg[1], out_q)

    qc.cx(log_q, out_q)

    cxxx(qc, out_q, stab_reg[4], stab_reg[5], log_q)

    qc.barrier()

In [5]:
def encode_with_five_qubit_code(qc, log_q, stab_reg):
    qc.h(stab_reg[0])
    qc.s(stab_reg[0])
    qc.cy(stab_reg[0], log_q)
    
    qc.h(stab_reg[1])
    qc.cx(stab_reg[1], log_q)
    
    qc.h(stab_reg[2])
    czz(qc, stab_reg[2], stab_reg[0], stab_reg[1])
    qc.cx(stab_reg[2], log_q)
        
    qc.h(stab_reg[3])
    qc.s(stab_reg[3])
    czz(qc, stab_reg[3], stab_reg[0], stab_reg[2])
    qc.cy(stab_reg[3], log_q)

    qc.barrier()

def five_qubit_code_measure_syndrome(qc, log_q, stab_reg, anc_reg, class_reg):
    qc.h(anc_reg)
    
    cxxzz(qc, anc_reg[0], stab_reg[0], stab_reg[3], stab_reg[1], stab_reg[2])
    cxxzz(qc, anc_reg[1], stab_reg[1], log_q, stab_reg[2], stab_reg[3])
    cxxzz(qc, anc_reg[2], stab_reg[0], stab_reg[2], stab_reg[3], log_q)
    cxxzz(qc, anc_reg[3], stab_reg[1], stab_reg[3], stab_reg[0], log_q)

    qc.h(anc_reg)
    qc.measure(anc_reg, class_reg)

    qc.barrier()

def five_qubit_code_correct_errors(qc, log_q, stab_reg, class_reg):
    # Define correction gates using a mapping of the table values.
    def apply_correction(circuit, correction, qubit):
        if correction == "X":
            circuit.x(qubit)
        elif correction == "Z":
            circuit.z(qubit)
        elif correction == "Y":
            circuit.y(qubit)
        elif correction == "I":
            circuit.i(qubit) #does nothing, just to be there

    corrections = {
        1: ["X", stab_reg[0]], 10: ["Z", stab_reg[0]], 11: ["Y", stab_reg[0]],
        8: ["X", stab_reg[1]], 5: ["Z", stab_reg[1]], 13: ["Y", stab_reg[1]],
        12: ["X", stab_reg[2]], 2: ["Z", stab_reg[2]], 14: ["Y", stab_reg[2]],
        6: ["X", stab_reg[3]], 9: ["Z", stab_reg[3]], 15: ["Y", stab_reg[3]],
        3: ["X", log_q],      4: ["Z", log_q],       7: ["Y", log_q]
    }


    # Iterate through all correction cases from the table.
    for decimal_value, correction_qubit in corrections.items():
        # Prepare the classical condition for the current value.
        condition = [int(x) for x in format(decimal_value, '04b')]
        control_ops = [
            qc.if_test((class_reg[i], value))
            for i, value in enumerate(condition)
        ]
        with control_ops[0]:
            with control_ops[1]:
                    with control_ops[2]:
                        with control_ops[3]:
                            apply_correction(qc, correction_qubit[0], correction_qubit[1])
    qc.barrier()
    
def decode_with_five_qubit_code(qc, log_q, stab_reg, out_q):
    qc.cx(stab_reg, out_q)
    qc.cx(log_q, out_q)
    czz(qc, out_q, stab_reg[0], stab_reg[3])
    qc.cx(out_q, log_q)

    qc.barrier()

In [6]:
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

backend_name = "ibm_brisbane"
backend = service.backend(backend_name)

#noise_model = NoiseModel.from_backend(backend)
#backend = AerSimulator(noise_model=noise_model, method="matrix_product_state")

#passmanager = generate_preset_pass_manager(
#    optimization_level=3, backend=backend
#)

<h3 style="color:white"> 
Control Group
</h4>

In [7]:
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
control_qc = QuantumCircuit(q,c)

#Bell State
control_qc.h(q[0])
control_qc.cx(q[0],q[1])

#Measure
control_qc.measure(q[0], c[0])
control_qc.measure(q[1], c[1])

#qc.draw("mpl", initial_state=True, scale=0.5, fold=-1, style="clifford", reverse_bits=False, plot_barriers=False, filename="Diagrams/control.png")

#control_noisy_circ = passmanager.run(qc)

#result = backend.run(noisy_circ, shots=8192).result()
#plot_histogram(result.get_counts(0), filename="Results/control.png")

<h3 style="color:white"> 
Shor's Error Correction Code
</h3>

In [8]:
q1 = QuantumRegister(1, 'q1')
q2 = QuantumRegister(1, 'q2')
c = ClassicalRegister(2,'c')
a1 = QuantumRegister(8, 'a1')
a2 = QuantumRegister(8, 'a2')
shor_qc = QuantumCircuit(q1,a1,q2,a2,c)

shor_qc.initialize(0, a1)
shor_qc.initialize(0, a2)

shor_qc.barrier()

#Bell State
shor_qc.h(q1[0])
shor_qc.cx(q1[0],q2[0])

shor_qc.barrier()

#Encode
encode_with_shors(shor_qc, q1, a1)
encode_with_shors(shor_qc, q2, a2)

#Decode
decode_with_shors(shor_qc, q1, c, a1)
decode_with_shors(shor_qc, q2, c, a2)

#Measure
shor_qc.measure(q1[0], c[0])
shor_qc.measure(q2[0], c[1])

#qc.draw("mpl", initial_state=True, scale=0.5, fold=-1, style="clifford", reverse_bits=False, plot_barriers=False, filename="Diagrams/shor.png")

#shor_noisy_circ = passmanager.run(qc)

#result = backend.run(noisy_circ, shots=8192).result()

#plot_histogram(result.get_counts(0), filename="Results/shor.png")



<h3 style="color:white"> 
Steane Error Correction Code
</h3>

In [9]:
q1 = QuantumRegister(1, 'log_qubit 1')
s1 = QuantumRegister(6, 'stabilizer 1')
o1 = QuantumRegister(1, 'output 1')
a1 = QuantumRegister(6, 'ancilla 1')
c1 = ClassicalRegister(6, 'measured errors 1')
r1 = ClassicalRegister(1, 'measured output 1')
q2 = QuantumRegister(1, 'log_qubit 2')
s2 = QuantumRegister(6, 'stabilizer 2')
o2 = QuantumRegister(1, 'output 2')
a2 = QuantumRegister(6, 'ancilla 2')
c2 = ClassicalRegister(6, 'measured errors 2')
r2 = ClassicalRegister(1, 'measured output 2')
steane_qc = QuantumCircuit(a1, s1, q1, a2, s2, q2, o1, o2, c1, c2, r1, r2)

#steane_qc.reset(range(steane_qc.num_qubits))

steane_qc.initialize(0, s1)
steane_qc.initialize(0, o1)
steane_qc.initialize(0, a1)
steane_qc.initialize(0, s2)
steane_qc.initialize(0, o2)
steane_qc.initialize(0, a2)

steane_qc.barrier()

steane_qc.h(q1[0])
steane_qc.cx(q1[0], q2[0])

steane_qc.barrier()

#Encode Qubits
encode_with_steane(steane_qc, q1[0], s1)
encode_with_steane(steane_qc, q2[0], s2)

#Measure Syndrome
steane_measure_syndrome(steane_qc, q1[0], s1, a1, c1)
steane_measure_syndrome(steane_qc, q2[0], s2, a2, c2)

#Correct Errors
steane_correct_errors(steane_qc, q1[0], s1, c1)
steane_correct_errors(steane_qc, q2[0], s2, c2)

#Decode Qubits
decode_with_steane(steane_qc, q1[0], s1, o1[0])
decode_with_steane(steane_qc, q2[0], s2, o2[0])

steane_qc.barrier()

#Measure Output
steane_qc.measure(o1[0], r1[0])
steane_qc.measure(o2[0], r2[0])

#steane_noisy_circ=passmanager.run(qc)

#qc.draw("mpl", style="clifford", scale=0.75, fold=-1, plot_barriers=True, initial_state=True, justify="none", filename="Diagrams/steane.png")

## Check circuit properties
#print("Depth:", qc.depth())
#print("Number of qubits:", qc.num_qubits)
#print("Number of gates:", qc.size())

In [10]:
#result = backend.run(noisy_circ, shots=8192).result()

In [11]:
#proc_results = {"0 0" : 0, "0 1" : 0, "1 0" : 0, "1 1" : 0}
#for res in result.get_counts(0):
#    proc_results[res[:3]] += result.get_counts(0)[res]
#plot_histogram(proc_results, filename="Results/steane.png")

<h3 style="color:white"> 
Five-qubit Error Correction Code
</h3>

In [12]:
q1 = QuantumRegister(1, 'log_qubit 1')
s1 = QuantumRegister(4, 'stabilizer 1')
o1 = QuantumRegister(1, 'output 1')
a1 = QuantumRegister(4, 'anicilla 1')
c1 = ClassicalRegister(4, 'measured errors 1')
r1 = ClassicalRegister(1, 'measured output 1')
q2 = QuantumRegister(1, 'log_qubit 2')
s2 = QuantumRegister(4, 'stabilizer 2')
o2 = QuantumRegister(1, 'output 2')
a2 = QuantumRegister(4, 'anicilla 2')
c2 = ClassicalRegister(4, 'measured errors 2')
r2 = ClassicalRegister(1, 'measured output 2')
fqc_qc = QuantumCircuit(a1, s1, q1, o1, a2, s2, q2, o2, c1, c2, r1, r2)

fqc_qc.initialize(0, s1)
fqc_qc.initialize(0, o1)
fqc_qc.initialize(0, s2)
fqc_qc.initialize(0, o2)

fqc_qc.barrier()

#Set up Bell State
fqc_qc.h(q1[0])
fqc_qc.cx(q1[0], q2[0])

fqc_qc.barrier()

#Encode Qubits
encode_with_five_qubit_code(fqc_qc, q1[0], s1)
encode_with_five_qubit_code(fqc_qc, q2[0], s2)

#Measure Syndrome
five_qubit_code_measure_syndrome(fqc_qc, q1[0], s1, a1, c1)
five_qubit_code_measure_syndrome(fqc_qc, q2[0], s2, a2, c2)

#Correct Errors
five_qubit_code_correct_errors(fqc_qc, q1[0], s1, c1)
five_qubit_code_correct_errors(fqc_qc, q2[0], s2, c2)

#Decode Qubits
decode_with_five_qubit_code(fqc_qc, q1[0], s1, o1[0])
decode_with_five_qubit_code(fqc_qc, q2[0], s2, o2[0])

fqc_qc.barrier()

#Measure
fqc_qc.measure(o1[0], r1[0])
fqc_qc.measure(o2[0], r2[0])

#noisy_circ=passmanager.run(fqc_qc)

#fqc_qc.draw("mpl", initial_state=True, scale=0.5, fold=-1, style="clifford", reverse_bits=False, plot_barriers=True, justify="none", filename="Diagrams/five_qubit_code.png")

In [13]:
#result = backend.run(noisy_circ, shots=8096).result()

In [14]:
#proc_results = {"0 0" : 0, "0 1" : 0, "1 0" : 0, "1 1" : 0}
#for res in result.get_counts(0):
#    proc_results[res[:3]] += result.get_counts(0)[res]
#plot_histogram(proc_results, filename="Results/five_qubit_code.png")

<h3 style="color:white"> 
    Running experiments
</h3>

In [15]:
# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
control_isa_circuit = pm.run(control_qc)
shor_isa_circuit = pm.run(shor_qc)
steane_isa_circuit = pm.run(steane_qc)
fqc_isa_circuit = pm.run(fqc_qc)

In [ ]:
from qiskit_ibm_runtime import Batch, SamplerV2

# Create a batch of jobs
with Batch(backend=backend):
    sampler = SamplerV2()
    
    #Run each circuit (maximum of three pending jobs, so I can only run these three, and then run the Five Qubit Code
    control_job = sampler.run(pubs=[control_isa_circuit], shots=1024)
    shor_job = sampler.run(pubs=[shor_isa_circuit], shots=1024)
    steane_job = sampler.run(pubs=[steane_isa_circuit], shots=1024)


print(f">>> Control Job ID: {control_job.job_id()}")
print(f">>> Shor Job ID: {shor_job.job_id()}")
print(f">>> Steane Job ID: {steane_job.job_id()}")

In [ ]:
#Get the results
control_result = control_job.result()
shor_result = shor_job.result()
steane_result = steane_job.result()

In [ ]:
sampler = SamplerV2(mode=backend)
fqc_job = sampler.run(pubs=[fqc_isa_circuit], shots=1024)
print(f">>> Five Qubit Code Job ID: {fqc_job.job_id()}")
fqc_result = fqc_job.result()